<a href="https://colab.research.google.com/github/bartlomiejandruchow/AI/blob/main/kerasAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras import layers


In [ ]:
# Seqential API

model = keras.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax'),
])

In [ ]:
model = keras.Sequential(name="my_example_model")

model.add(keras.Input(shape=(3,)))
model.add(layers.Dense(64, activation="relu", name="my_first_layer"))
#model.build(input_shape=(None,3))
model.summary()


In [ ]:
model.add(layers.Dense(10, activation="softmax", name="my_last_layer"))
model.summary()

#model.weights

In [ ]:
# Functional API

inputs = keras.Input(shape=(3,))
print(inputs.dtype)

features = layers.Dense(64, activation="relu")(inputs)
outputs = layers.Dense(10, activation="softmax")(features)
model = keras.Model(inputs=inputs, outputs=outputs, name="my_functional_model")

model.summary()


In [ ]:
# Subclassing Model class
import keras
from keras import layers


class CustomerTicketModel(keras.Model):
  def __init__(self, num_departments):
    super.__init__()
    self.concat_layer = layers.Concatenate()
    self.mixing_layer = layers.Dense(64, activation="relu")
    self.priority_scorer = layers.Dense(1, activation="sigmoid")
    self.department_classifier = layers.Dense(num_departments, activation="softmax")

  def call(self, inputs):
    title = inputs["title"]
    text_body = inputs["text_body"]
    tags = inputs["tags"]

    features = self.concat_layer([title, text_body, tags])
    features = self.mixing_layer(features)
    priority = self.priority_scorer(features)
    department = self.department_classifier(features)
    return priority, department


model = CustomerTicketModel(num_departments=4)
pririty, department = model({"title":title_data,"text_body":text_body_data, "tags":tags_data })


In [ ]:
from keras.datasets import mnist
import keras
from keras import layers

def get_mnist_model():
  inputs = keras.Input(shape=(28 * 28,))
  features = layers.Dense(512, activation="relu")(inputs)
  features = layers.Dropout(0.5)(features)
  outputs = layers.Dense(10, activation="softmax")(features)
  model = keras.Model(inputs, outputs)
  return model

(images,labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28*28)).astype("float32") / 255

train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

print(train_images.shape)
print(train_labels.shape)
print(train_labels[0:10])


In [ ]:
model = get_mnist_model()
model.compile(
    optimizer="rmsprop",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"],
)

model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_data=(val_images, val_labels))

test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

In [ ]:
from keras import ops
print(predictions[0])
print(test_labels[0])
print(ops.one_hot(test_labels[0], num_classes=10))

In [ ]:
from keras import ops

class RootMeanSquaredError(keras.metrics.Metric):
  def __init__(self, name="rmse",  **kwargs):
    super().__init__(name=name, **kwargs)
    self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
    self.total_samples = self.add_weight(name="total_samples", initializer="zeros")

  def update_state(self, y_true, y_pred, sample_weight = None):
    y_true = ops.one_hot(y_true, num_classes=ops.shape(y_pred)[1])
    mse = ops.sum(ops.square(y_true - y_pred))
    self.mse_sum.assign_add(mse)
    num_samples = ops.shape(y_pred)[0]
    self.total_samples.assign_add(num_samples)
    # In classification settings, if y_true is an integer class (sparse label)
    # , you often convert it into a one-hot vector to compare it with y_pred
    # (which is usually probabilities or logits after softmax).

  def result(self):
    return ops.sqrt(self.mse_sum / self.total_samples)

  def reset_state(self):
    self.mse_sum.assign(0.)
    self.total_samples.assign(0.)


In [ ]:
model = get_mnist_model()
model.compile(
    optimizer="adam",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy", RootMeanSquaredError()],
    )

model.fit(train_images
          , train_labels
          , epochs=3
          , batch_size=64
          , validation_data=(val_images, val_labels)
          ,
          )

In [ ]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="accuracy",
        patience=1,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True,
    ),
]

model = get_mnist_model()
model.compile(
    optimizer="adam",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"],
    )

model.fit(train_images
          , train_labels
          , epochs=10
          , callbacks = callbacks_list
          , validation_data=(val_images, val_labels)
          ,
          )

In [ ]:
model = keras.models.load_model("checkpoint_path.keras")
test_metrics = model.evaluate(test_images, test_labels)

In [ ]:
from matplotlib import pyplot as plt

class LossHistory(keras.callbacks.Callback):
  def on_train_begin(self, logs):
    self.per_batch_losses = []

  def on_batch_end(self, batch, logs):
    self.per_batch_losses.append(logs.get("loss"))

  def on_epoch_end(self, epoch, logs):
    plt.clf()
    plt.plot(
        range(len(self.per_batch_losses)),
        self.per_batch_losses,
        label="Training loss for each batch"
             )
    plt.savefig(f"plot_at_epoch_{epoch}", dpi=300)
    self.per_batch_losses = []



In [ ]:
model = get_mnist_model()
model.compile(
    optimizer="adam",
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"],
    )

model.fit(train_images
          , train_labels
          , epochs=10
          , callbacks = [LossHistory()]
          , validation_data=(val_images, val_labels)
          ,
          )